In [23]:
import sys

import matplotlib.pyplot as plt
import numpy as np

from luxai_s2.env import LuxAI_S2
from luxai_s2.state import ObservationStateDict, StatsStateDict

from lux.kit import obs_to_game_state, GameState, EnvConfig
from lux.utils import direction_to, my_turn_to_place_factory

In [66]:
env = LuxAI_S2() # create the environment object
obs = env.reset(seed=41) # resets an environment with a seed

In [67]:
# Rule-based agent
class Agent():
    def __init__(self, player: str, env_cfg: EnvConfig) -> None:
        self.player = player
        self.opp_player = "player_1" if self.player == "player_0" else "player_0"
        np.random.seed(0)
        self.env_cfg: EnvConfig = env_cfg

    def early_setup(self, step: int, obs, remainingOverageTime: int = 60):
        if step == 0:
            # bid 0 to not waste resources bidding and declare as the default faction
            # you can bid -n to prefer going second or n to prefer going first in placement
            return dict(faction="AlphaStrike", bid=0)
        else:
            game_state = obs_to_game_state(step, self.env_cfg, obs)
            # factory placement period
            
            # how much water and metal you have in your starting pool to give to new factories
            water_left = game_state.teams[self.player].water
            metal_left = game_state.teams[self.player].metal
            
            # how many factories you have left to place
            factories_to_place = game_state.teams[self.player].factories_to_place
            # whether it is your turn to place a factory
            my_turn_to_place = my_turn_to_place_factory(game_state.teams[self.player].place_first, step)
            if factories_to_place > 0 and my_turn_to_place:
                # we will spawn our factory in a random location with 150 metal and water if it is our turn to place
                potential_spawns = np.array(list(zip(*np.where(obs["board"]["valid_spawns_mask"] == 1))))
                spawn_loc = potential_spawns[np.random.randint(0, len(potential_spawns))]
                return dict(spawn=spawn_loc, metal=150, water=150)
            return dict()

    def act(self, step: int, obs, remainingOverageTime: int = 60):
        actions = dict()
        game_state = obs_to_game_state(step, self.env_cfg, obs)
        factories = game_state.factories[self.player]
        for unit_id, factory in factories.items():
            if factory.power >= self.env_cfg.ROBOTS["HEAVY"].POWER_COST and \
            factory.cargo.metal >= self.env_cfg.ROBOTS["HEAVY"].METAL_COST:
                actions[unit_id] = factory.build_heavy()
                
        # iterate over our units and have them mine the closest ice tile
        units = game_state.units[self.player]
        ice_map = game_state.board.ice # flip the board as it stores by rows then columns
        ice_tile_locations = np.argwhere(ice_map == 1) # numpy magic to get the position of every ice tile
        for unit_id, unit in units.items():
            # compute the distance to each ice tile from this unit and pick the closest
            ice_tile_distances = np.mean((ice_tile_locations - unit.pos) ** 2, 1)
            closest_ice_tile = ice_tile_locations[np.argmin(ice_tile_distances)]
            
            # if we have reached the ice tile, start mining if possible
            if np.all(closest_ice_tile == unit.pos):
                if unit.power >= unit.dig_cost(game_state) + unit.action_queue_cost(game_state):
                    actions[unit_id] = [unit.dig(repeat=0)]
            else:
                direction = direction_to(unit.pos, closest_ice_tile)
                move_cost = unit.move_cost(game_state, direction)
                # check move_cost is not None, meaning that direction is not off the map or blocked
                # check if unit has enough power to move in addition to updating the action queue.
                if move_cost is not None and unit.power >= move_cost + unit.action_queue_cost(game_state):
                    actions[unit_id] = [unit.move(direction, repeat=0)]
            # since we are using the simple embedded visualizer, we will have to print out details about units
            # importantly, note that we print with file=sys.stderr. Printing with anything will cause your agent to fail
            if unit.cargo.ice > 50:
                print(game_state.real_env_steps, unit, f"has {unit.cargo.ice} ice", file=sys.stderr)
        return actions

In [39]:
def interact(env, agents, steps):
    # reset our env
    obs = env.reset(seed=41)
    np.random.seed(0)
    imgs = []
    step = 0
    # Note that as the environment has two phases, we also keep track a value called 
    # `real_env_steps` in the environment state. The first phase ends once `real_env_steps` is 0 and used below

    # iterate until phase 1 ends
    while env.state.real_env_steps < 0:
        if step >= steps: break
        actions = {}
        for player in env.agents:
            o = obs[player]
            a = agents[player].early_setup(step, o)
            actions[player] = a
        step += 1
        obs, rewards, dones, infos = env.step(actions)
        imgs += [env.render("rgb_array", width=640, height=640)]
    done = False
    while not done:
        if step >= steps: break
        actions = {}
        for player in env.agents:
            o = obs[player]
            a = agents[player].act(step, o)
            actions[player] = a
        step += 1
        obs, rewards, dones, infos = env.step(actions)
        imgs += [env.render("rgb_array", width=640, height=640)]
        done = dones["player_0"] and dones["player_1"]
    return obs

In [68]:
agents = {player: Agent(player, env.state.env_cfg) for player in env.agents}
last_obs = interact(env, agents, steps=100)

7 [0] unit_4 HEAVY at [16  4] has 60 ice
8 [0] unit_4 HEAVY at [16  4] has 80 ice
9 [0] unit_4 HEAVY at [16  4] has 100 ice
9 [0] unit_5 HEAVY at [45 39] has 60 ice
10 [0] unit_4 HEAVY at [16  4] has 120 ice
10 [0] unit_5 HEAVY at [45 39] has 80 ice
11 [0] unit_4 HEAVY at [16  4] has 120 ice
11 [0] unit_5 HEAVY at [45 39] has 80 ice
12 [0] unit_4 HEAVY at [16  4] has 120 ice
12 [0] unit_5 HEAVY at [45 39] has 80 ice
13 [0] unit_4 HEAVY at [16  4] has 120 ice
13 [0] unit_5 HEAVY at [45 39] has 80 ice
14 [0] unit_4 HEAVY at [16  4] has 120 ice
14 [0] unit_5 HEAVY at [45 39] has 80 ice
15 [0] unit_4 HEAVY at [16  4] has 140 ice
15 [0] unit_5 HEAVY at [45 39] has 100 ice
16 [0] unit_4 HEAVY at [16  4] has 140 ice
16 [0] unit_5 HEAVY at [45 39] has 100 ice
17 [0] unit_4 HEAVY at [16  4] has 140 ice
17 [0] unit_5 HEAVY at [45 39] has 100 ice
18 [0] unit_4 HEAVY at [16  4] has 140 ice
18 [0] unit_5 HEAVY at [45 39] has 100 ice
19 [0] unit_4 HEAVY at [16  4] has 140 ice
19 [0] unit_5 HEAVY at 

In [70]:
game_state = obs_to_game_state(101, env.state.env_cfg, last_obs['player_0'])

In [72]:
factories = game_state.factories['player_0']

In [73]:
factories

{'factory_0': Factory(team_id=0, unit_id='factory_0', strain_id=0, power=5250, cargo=UnitCargo(ice=0, ore=0, water=55, metal=50), pos=array([18,  4]), env_cfg=EnvConfig(max_episode_length=1000, map_size=48, verbose=1, validate_action_space=True, max_transfer_amount=3000, MIN_FACTORIES=2, MAX_FACTORIES=5, CYCLE_LENGTH=50, DAY_LENGTH=30, UNIT_ACTION_QUEUE_SIZE=20, MAX_RUBBLE=100, FACTORY_RUBBLE_AFTER_DESTRUCTION=50, INIT_WATER_METAL_PER_FACTORY=150, INIT_POWER_PER_FACTORY=1000, MIN_LICHEN_TO_SPREAD=20, LICHEN_LOST_WITHOUT_WATER=1, LICHEN_GAINED_WITH_WATER=1, MAX_LICHEN_PER_TILE=100, POWER_PER_CONNECTED_LICHEN_TILE=1, LICHEN_WATERING_COST_FACTOR=10, BIDDING_SYSTEM=True, FACTORY_PROCESSING_RATE_WATER=100, ICE_WATER_RATIO=4, FACTORY_PROCESSING_RATE_METAL=50, ORE_METAL_RATIO=5, FACTORY_CHARGE=50, FACTORY_WATER_CONSUMPTION=1, POWER_LOSS_FACTOR=0.5, ROBOTS={'LIGHT': UnitConfig(METAL_COST=10, POWER_COST=50, CARGO_SPACE=100, BATTERY_CAPACITY=150, CHARGE=1, INIT_POWER=50, MOVE_COST=1, RUBBLE_MOVE

In [69]:

stats: StatsStateDict = env.state.stats['player_0']

KeyError: 'player_0'

In [65]:
test.stats

{}